In [1]:
%%capture
!git clone https://github.com/Microsoft/dowhy.git
!sudo apt install graphviz libgraphviz-dev graphviz-dev pkg-config
!pip install pygraphviz \
 --install-option="--include-path=/usr/include/graphviz" \
 --install-option="--library-path=/usr/lib/graphviz/"
!pip install -r ./dowhy/requirements.txt
!pip install --upgrade pandas
!pip install --upgrade statsmodels
!python ./dowhy/setup.py install
!wget -P ./dowhy/dowhy/causal_estimators/ \
https://raw.githubusercontent.com/ckbjimmy/2019_tokyo/master/supp/logistic_regression_estimator.py
!pip install tableone

In [2]:
import sys
sys.path.append('dowhy')
import random
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
from scipy import stats
from IPython.display import Image, display

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import tableone
from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [4]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/SCCM Datathon 2023/Code

/content/drive/MyDrive/SCCM Datathon 2023/Code


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Get data

In [6]:
df = pd.read_csv('Data/Final_Cohort.csv')
df

Output hidden; open in https://colab.research.google.com to view.

In [7]:
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Unnamed: 0', 'stay_id', 'subject_id', 'starttime', 'endtime', 'gender',
       'race', 'language', 'insurance', '221385', '221668', '221744', '221833',
       '222168', '225150', '225154', '229420'],
      dtype='object')

In [8]:
columns = ['gender', 'race', 'language', 'insurance', '221385', '221668', '221744', '221833',
       '222168', '225150', '225154', '229420']
df = df[columns]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df

Output hidden; open in https://colab.research.google.com to view.

In [10]:
categories = ['white', 'black', 'asian', 'hispanic', 'other']

def get_race(race):
  for category in categories:
    if category in race.lower():
      return category
  return 'other'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
df.gender = df.gender.replace({'F':0, 'M':1})
df.language = df.language.replace({'ENGLISH':1, '?':0})
df.insurance = df.insurance.replace({'Medicare':0, 'Other':1, 'Medicaid':2})
df.race = df.race.apply(get_race)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-11-7473e0805694>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.gender = df.gender.replace({'F':0, 'M':1})
<ipython-input-11-7473e0805694>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [12]:
# Convert 'Category' column to dummy variables
dummy_df = pd.get_dummies(df['race'])

# Concatenate the dummy DataFrame with the original DataFrame
df = pd.concat([df, dummy_df], axis=1)

df.drop(columns=['race'], inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
for category in categories:
  df[category] = df[category].replace({False:0, True:1})

df

Output hidden; open in https://colab.research.google.com to view.

In [14]:
df.sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


gender       4233
language     6529
insurance    4664
221385         24
221668       3357
221744        637
221833        346
222168       6028
225150       1375
225154        838
229420        975
asian         193
black         733
hispanic      251
other        1600
white        4552
dtype: int64

In [15]:
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['gender', 'language', 'insurance', '221385', '221668', '221744',
       '221833', '222168', '225150', '225154', '229420', 'asian', 'black',
       'hispanic', 'other', 'white'],
      dtype='object')

### Causal Modeling

### Black

In [16]:
#!pip install CausalInference
#!git clone https://github.com/Microsoft/dowhy.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
from causalinference import CausalModel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
Y = df['222168'].to_numpy()
D = df['black'].to_numpy()
X = df[['gender', 'language', 'insurance']].to_numpy()

causal = CausalModel(Y, D, X)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
print(causal.summary_stats)

# Estimate the treatment effect using ordinary least squares (OLS)
causal.est_via_ols()

# Print the estimated treatment effect
print("Estimated ATE:", causal.estimates)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]



Summary Statistics

                      Controls (N_c=6596)         Treated (N_t=733)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        0.826        0.379        0.794        0.405       -0.032

                      Controls (N_c=6596)         Treated (N_t=733)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.591        0.492        0.456        0.498       -0.274
             X1        0.892        0.311        0.881        0.324       -0.033
             X2        0.630        0.635        0.693        0.675        0.096

Estimated ATE: 
Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
-------------------------------------

### White

In [20]:
Y = df['222168'].to_numpy()
D = df['white'].to_numpy()
X = df[['gender', 'language', 'insurance']].to_numpy()

causal = CausalModel(Y, D, X)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
print(causal.summary_stats)

# Estimate the treatment effect using ordinary least squares (OLS)
causal.est_via_ols()

# Print the estimated treatment effect
print("Estimated ATE:", causal.estimates)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]



Summary Statistics

                      Controls (N_c=2777)        Treated (N_t=4552)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        0.826        0.379        0.820        0.384       -0.006

                      Controls (N_c=2777)        Treated (N_t=4552)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.572        0.495        0.581        0.493        0.017
             X1        0.782        0.413        0.957        0.203        0.536
             X2        0.736        0.665        0.576        0.617       -0.249

Estimated ATE: 
Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
-------------------------------------

### hispanic

In [22]:
Y = df['222168'].to_numpy()
D = df['hispanic'].to_numpy()
X = df[['gender', 'language', 'insurance']].to_numpy()

causal = CausalModel(Y, D, X)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
print(causal.summary_stats)

# Estimate the treatment effect using ordinary least squares (OLS)
causal.est_via_ols()

# Print the estimated treatment effect
print("Estimated ATE:", causal.estimates)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]



Summary Statistics

                      Controls (N_c=7078)         Treated (N_t=251)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        0.821        0.384        0.876        0.330        0.056

                      Controls (N_c=7078)         Treated (N_t=251)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        0.577        0.494        0.606        0.490        0.059
             X1        0.907        0.290        0.426        0.496       -1.185
             X2        0.626        0.634        0.920        0.744        0.425

Estimated ATE: 
Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
-------------------------------------